# Brightway2 seminar
Chris Mutel ([PSI](https://www.psi.ch/)), Pascal Lesage ([CIRAIG](http://www.ciraig.org/en/))

## Day 1, afternoon
### Session on data input/output

### Learning objectives  
  - Learn how to input LCI data to Brightway in different ways:  
    - Programmatically, via dictionaries created directly in Python  
    - Excel/cvs importers  
    - SimaPro csv
      - Two unit-process example
      - Agribalyse
    - Importing ecoSpold, ecoSpold2

## Standard inputs and setup

In [2]:
import brightway2 as bw

Setting the project

In [3]:
bw.projects.set_current('bw2_seminar_2017')

Assigning our database to a variable

In [9]:
eidb = bw.Database('ecoinvent 2.2')

## Context

Performing LCA generally requires:
  - Background LCI data (e.g. an LCI database)  
  - Foreground LCI data  
  - Sets of characterization factors.    
This section will deal with the way data is input to Brightway

Useful documentation about what a database in Brightway is can be found [here](http://nbviewer.jupyter.org/urls/bitbucket.org/cmutel/brightway2/raw/default/notebooks/Databases.ipynb) and [here](https://docs.brightwaylca.org/intro.html#inventory-databases).

## 1) Creating a database programmatically

One can create a database via a database. This database would include activities that would contain information about the activities themselves as well as information about the exchanges that are output to this activity (although these get seperated out when the data is written to the actual `database.db`). Let's look at the components of a database for a random activity:

In [14]:
random_act = eidb.random()

In [15]:
# Information about activities themselves:
random_act.as_dict()

{'authors': [{'address': 'Basler und Hofmann AG, 8032 Zuerich',
   'company': 'B+H',
   'country': 'CH',
   'email': 'alex.primas@bhz.ch',
   'name': 'Alex Primas'}],
 'categories': ['heat pumps', 'heating systems'],
 'code': '68f3b75b012a7865cd02397f271faf1f',
 'comment': 'NOx and CO emissions derived from measurements of gas boilers operated with natural gas under controlled conditions; no adjustment to real operation of the heat pump system is made due to lack of information. Other emission approximised with data for gas boilers from different references.\nThe module includes input biogas input from low pressure gas network (CH), infrastructure (heat pump with peak boiler and borehole heat exchanger), emissions to air and water, and electricity needed for operation.\nLocation:  Natural gas input modelled for Switzerland. Process applicable in central European conditions.\nTechnology:  Diffusion absorption heat pump with 4 kWth nominal thermal power connected to a short borehole heat

In [16]:
# Information associated with the exchanges of this activity:
[exc for exc in random_act.exchanges()][1].as_dict()

{'amount': 0.75758,
 'categories': ('biomass', 'fuels'),
 'comment': '(3,4,2,2,1,5); uncertainty of operation temperatures of source and supply',
 'input': ('ecoinvent 2.2', '64cfe2b1ae2fc743f08467aa19650182'),
 'loc': -0.27762613661395941,
 'location': 'CH',
 'name': 'methane, 96 vol-%, from biogas, low pressure, at consumer',
 'negative': False,
 'output': ('ecoinvent 2.2', '68f3b75b012a7865cd02397f271faf1f'),
 'scale': 0.11702196289124893,
 'type': 'technosphere',
 'uncertainty type': 2,
 'unit': 'megajoule'}

Let's create one for our simple LCA:

In [34]:
database_as_dictionary = bw.Database("Database as dict")

water_bottle_data = {
    ("Database as dict", "Some code for the bottle production"): {
        "name": "Water bottle production",
        'unit': 'unit',
        'location': 'CH',
        'categories': ("Some made up", "category here"),
        "exchanges": [{
            "amount": 0.33,
            "input": ('ecoinvent 2.2', 'c028a331b2ce1cd30ce326c3ba284a62'), #Aluminium
            "type": "technosphere",
            "uncertainty type":0,
            "unit=": "kg"},
                      {
            "amount": 0.33,
            "input": ('ecoinvent 2.2', 'ea5e562ab50994b90b7b3e0a3cd0e498'), #Deformation stroke
            "type": "technosphere",
            "uncertainty type":0,
            "unit=": "kg"}
        ],
        },
    ("Database as dict", "Some code for drinking a bottle full of water"): {
        "name": "Water drinking",
        'unit': 'liter',
        'location': 'CH',
        'categories': ("Another made up", "category here"),
        "exchanges": [{
            "amount": 1,
            "input": ("Database as dict", "Some code for the bottle production"), #Our water bottle
            "type": "technosphere",
            "uncertainty type":5,
            "loc":0.005,
            "min":0.0005,
            "max":0.05,
            "unit": "kg"},
                      {
            "amount": 1,
            "input": ('ecoinvent 2.2', 'b80ba272db9a3202e07a29d7574cff80'), #Water
            "type": "technosphere",
            "uncertainty type":0,
            "unit=": "kg"}],
        }
}

In [35]:
database_as_dictionary.write(water_bottle_data)

Writing activities to SQLite3 database:
0%  100%
[##] | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 03/29/2017 00:15:46
  Finished: 03/29/2017 00:15:46
  Total time elapsed: 00:00:00
  CPU %: 104.20
  Memory %: 1.60


In [36]:
len(database_as_dictionary)

2

In [37]:
drinking_act = [act for act in database_as_dictionary if "drinking" in act['name']][0]
drinking_act

'Water drinking' (liter, CH, ('Another made up', 'category here'))

In [38]:
water_bottle_LCA_from_dict_input = bw.LCA({drinking_act:1}, ('IPCC 2013', 'climate change', 'GWP 100a'))

KeyboardInterrupt: 

In [ ]:
water_bottle_LCA_from_dict_input.lci()
water_bottle_LCA_from_dict_input.lcia()
water_bottle_LCA_from_dict_input.score

# Chris - not solving...

## Approach 2: from an Excel spreadsheet

See spreadsheet on Github.

In [40]:
from os.path import join

In [41]:
my_dir = r'C:\Users\pasca\Desktop'
imp = bw.ExcelImporter(join(my_dir, "excel_importer_example.xlsx"))
imp.apply_strategies()
imp.match_database("ecoinvent 2.2", fields=('name', 'unit', 'location'))
imp.statistics()

Extracted 3 worksheets in 0.10 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applied 14 strategies in 0.87 seconds
Applying strategy: link_iterable_by_fields
2 datasets
6 exchanges
5 unlinked exchanges
  Type production: 2 unique unlinked exchanges
  Type technosphere: 3 unique unlinked exchanges


(2, 6, 5)

# Failure #2!

## Approach 3: From SimaPro csv

AgriBalyse
ecoSpold1
ecoSpold2